In [1]:
from subprocess import Popen,PIPE

stack='s3://mousebraindata-open/MD657/'

In [2]:
import re
from os import system
import socket

def get_file_table(path):
    system("aws s3 ls %s > awsfiles.txt"%path)
    pat=re.compile(r'(.*lossless)(.*)$')

    T={}
    with open('awsfiles.txt','r') as files:
        for file in files.readlines():
            parts=file.strip().split()
            if len(parts)!=4:
                continue
            filename=parts[3]
            if not 'lossless' in filename:
                continue
            m=pat.match(filename)
            if m:
                file,ext= m.groups()
                info=(ext,parts[1:3])
                if file in T:
                    T[file].append(info)
                else:
                    T[file]=[info]
            else:
                print(filename,'no match')
    return T


In [3]:
T=get_file_table(stack)
!grep patches awsfiles.txt |wc

     463    1852   43931


In [96]:
!grep MD657-F3-2017.02.17-18.07.23_MD657_1_0007_lossless_patches.tgz awsfiles.txt

2018-09-14 23:49:03  160281922 MD657-F3-2017.02.17-18.07.23_MD657_1_0007_lossless_patches.tgz


In [69]:
!grep .lock-ip-172-30-1-171 awsfiles.txt

2018-09-14 21:45:09         73 MD657-F88-2017.02.23-17.31.36_MD657_2_0263_lossless.lock-ip-172-30-1-171
2018-09-14 21:47:06         73 MD657-N96-2017.02.22-20.28.14_MD657_2_0287_lossless.lock-ip-172-30-1-171


In [70]:
!grep MD657-F88-2017.02.23-17.31.36_MD657_2_0287_lossless awsfiles.txt

In [4]:
T=get_file_table(stack)
!grep patches awsfiles.txt | wc
!grep less.jp2 awsfiles.txt | wc

     463    1852   43931
     463    1852   40227


In [35]:
!grep patches awsfiles.txt |wc

     442    1768   41937


In [5]:
from collections import Counter
T=get_file_table(stack)
C=Counter()
for item in T.items():
    extensions=[x for x in item[1]]
    for ext in extensions:
        if 'lock' in ext[0]:
            C[ext[0]]+=1
C

Counter({'.lock-ip-172-30-1-84': 89,
         '.lock-ip-172-30-1-39': 38,
         '.lock-ip-172-30-1-4': 38,
         '.lock-ip-172-30-1-66': 40,
         '.lock-ip-172-30-1-38': 33,
         '.lock-ip-172-30-1-44': 37,
         '.lock-ip-172-30-1-156': 36,
         '.lock-ip-172-30-1-126': 37,
         '.lock-ip-172-30-1-211': 41,
         '.lock-ip-172-30-1-183': 38,
         '.lock-ip-172-30-1-150': 36})

In [6]:
def run(command):
    print('system cmd=',command)
    system(command)

i=0
for item in T.items():
    #print(item[0])
    extensions=[x for x in item[1]]
    has_lock=False
    is_done=False
    is_small=False
    for ext in extensions:
        #print('++',ext)
        if ('patches'in ext[0]):
            if (int(ext[1][1])>1000):
                is_done=True
            else:
                is_small=True
                patch_name=item[0]+ext[0]
        
        if ('lock' in ext[0]):
            has_lock=True
            lock_name=item[0]+ext[0]

    if has_lock and not is_done:
        print('>>>>>>>>>>>>>>>>>>>>>>>> delete lock',lock_name)
        run('aws s3 rm %s%s'%(stack,lock_name))
    if is_small:
        print('====================== delete small patch',patch_name)
        run('aws s3 rm %s%s'%(stack,patch_name))
        
    #if i>10:
    #    break
    i+=1